# Australian Open 2020: Winner Predictor
## Web Intelligence Course, Ca' Foscari University, A.Y. 2019/2020
#### Diego Chinellato, 867637 - Giorgia Campardo, 867928

In [31]:
%matplotlib notebook

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, classification_report

In [94]:
data_types = {
    'ATP': int,
    'Location': object,
    'Tournament': object,
    'Series': object,
    'Court': object,
    'Surface': object,
    'Round': object,
    'Best of': int,
    'Winner': object,
    'Loser': object,
    'WRank': float,
    'LRank': float,
    'WPts': float,
    'LPts': float,
    'W1': float,
    'L1': float,
    'W2': float,
    'L2': float,
    'W3': float,
    'L3': float,
    'W4': float,
    'L4': float,
    'W5': float,
    'L5': float,
    'Wsets': float,
    'Lsets': float,
    'Comment': object,
    'B365W': float,
    'B365L': float,
    'EXW': object,
    'EXL': float,
    'LBW': float,
    'LBL': float,
    'PSW': float,
    'PSL': float,
    'SJW': float,
    'SJL': float,
    'MaxW': float,
    'MaxL': float,
    'AvgW': float,
    'AvgL': float,
    'WElo': float,
    'WSurfElo': float,
    'WHand': object,
    'WBHand': float,
    'LElo': float,
    'LSurfElo': float,
    'LHand': object,
    'LBHand': float
}
x = pd.read_csv('data/dataset.csv', 
                encoding='utf-8-sig', 
                dtype=data_types,
                parse_dates=['Date', 'WBD', 'LBD'])

In [95]:
print(x['WHand'].unique())
print(x['LHand'].unique())
print(x['WBHand'].unique())
print(x['LBHand'].unique())

['L' 'R' nan]
[nan 'R' 'L']
[ 1.  2. nan]
[nan  2.  1.]


In [78]:
def compute_elo_rankings(data):
    """
    Given the list on matches in chronological order, for each match, computes 
    the elo ranking of the 2 players at the beginning of the match
    """
    print("Elo rankings computing...")
    players=list(pd.Series(list(data.Winner)+list(data.Loser)).value_counts().index)
    elo=pd.Series(np.ones(len(players))*1500,index=players)
    ranking_elo=[(1500,1500)]
    for i in range(1,len(data)):
        w=data.iloc[i-1,:].Winner
        l=data.iloc[i-1,:].Loser
        elow=elo[w]
        elol=elo[l]
        pwin=1 / (1 + 10 ** ((elol - elow) / 400))    
        K_win=32
        K_los=32
        new_elow=elow+K_win*(1-pwin)
        new_elol=elol-K_los*(1-pwin)
        elo[w]=new_elow
        elo[l]=new_elol
        ranking_elo.append((elo[data.iloc[i,:].Winner],elo[data.iloc[i,:].Loser])) 
    ranking_elo=pd.DataFrame(ranking_elo,columns=["elo_winner","elo_loser"])    
    ranking_elo["proba_elo"]=1 / (1 + 10 ** ((ranking_elo["elo_loser"] - ranking_elo["elo_winner"]) / 400))   
    return ranking_elo

In [114]:
def preprocess_data(max_date=2014,
                    features_to_drop=[], 
                    missing_values="drop", 
                    drop_first=False):
    """
    Processes raw data and returns a tuple (X, Y) where X is the cleaned dataset and Y is the array of labels.
    """
    if max_date > 2019 or max_date < 2011:
        raise ValueError("Wrong date parameter")
    df = pd.read_csv("data/" + str(max_date) + ".csv", encoding='utf-8-sig')
    for year in range (max_date + 1, 2020):
        filename = "data/" + str(year) + ".csv"
        df = pd.concat((df, pd.read_csv(filename, encoding='utf-8-sig', dtype=data_types)))
    
    # Sort by date to calculate ELO
    X = df.sort_values(by='Date')
    
    # Drop unuseful columns
    features_to_drop += ['ATP', 'Location', 'Tournament', 'Date', 'Comment', 
                         'WPts', 'LPts', 'Wsets', 'Lsets', 
                         'W1', 'L1', 'W2', 'L2', 'W3', 'L3', 'W4', 'L4', 'W5', 'L5', 
                         'B365W', 'B365L', 'EXW', 'EXL', 'LBW', 'LBL', 'PSW', 'PSL', 'SJW', 'SJL',
                         'WBD', 'LBD']
    X = X.drop(columns=features_to_drop)
    
    # Deal with missing values
    X['WRank'] = X['WRank'].fillna(value=X['WRank'].max()+100).astype(int)
    X['LRank'] = X['LRank'].fillna(value=X['LRank'].max()+100).astype(int)

    if missing_values == 'drop':
        X = X.dropna()
    elif missing_values == 'custom':
        pass
    else:
        raise ValueError('Wrong parameter: missing_values')

    # Convert ordinal features to int (higher value means more important)
    series = ['ATP250', 'ATP500', 'Masters 1000', 'Masters Cup', 'Grand Slam']
    series2int = {s: i for i, s in enumerate(series)}
    rounds2int = {'1st Round': 0,
                  '2nd Round': 1,
                  '3rd Round': 2,
                  '4th Round': 3,
                  'Round Robin': 4,
                  'Quarterfinals': 5,
                  'Semifinals': 6,
                  'The Final': 7,
                 }
    X = X.replace({'Round': rounds2int, 'Series': series2int})
    
    # Convert categorical (binary) fields to int
    X = X.replace({'Court': {'Outdoor': 0, 'Indoor': 1}, 
                   'WHand': {'R': 0, 'L': 1}, 
                   'LHand': {'R': 0, 'L': 1}})
    X.astype({'WBHand': int, 'LBHand': int})
    
    # One hot encode categorical features into binary features
    X = pd.get_dummies(X, prefix=['Surface_'], columns=['Surface'], drop_first=drop_first)
    
    # Convert players to numeric ?
    players = set(X['Winner']) | set(X['Loser'])
    players_to_id = {}
    for i, player in enumerate(players):
        players_to_id[player] = i
    X = X.replace({'Winner': players_to_id, 'Loser': players_to_id})
    X = X.astype({'Winner': int, 'Loser': int})

    X = X.rename(columns={'Winner':'1st Player', 'Loser':'2nd Player', 
                          'WRank':'P1Rank', 'LRank':'P2Rank', 
                          'MaxW':'MaxP1', 'MaxL':'MaxP2', 
                          'AvgW':'AvgP1', 'AvgL':'AvgP2'})
    
    # Generate labels
    Y = np.concatenate([np.ones(X.shape[0], dtype=int), np.zeros(X.shape[0], dtype=int)])
    # Swap columns and concatenate to data
    tmp = X.copy()
    cols_to_swap = ['1st Player', '2nd Player', 'P1Rank', 'P2Rank', 'MaxP1', 'MaxP2',  'AvgP1',  'AvgP2',
                    'WElo', 'LElo', 'WSurfElo', 'LSurfElo', 'WHand', 'LHand', 'WBHand', 'LBHand']
    cols_swapped = ['2nd Player', '1st Player', 'P2Rank', 'P1Rank', 'MaxP2', 'MaxP1',  'AvgP2',  'AvgP1', 
                    'LElo', 'WElo', 'LSurfElo', 'WSurfElo', 'LHand', 'WHand', 'LBHand', 'WBHand']

    tmp[cols_to_swap] = tmp[cols_swapped]
    tmp.index = np.array(range(X.shape[0] + 1, X.shape[0] * 2 + 1))
    X = pd.concat((X, tmp))
    
    return X, Y

In [136]:
X, Y = preprocess_data(max_date=2018)
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6684 entries, 0 to 6684
Data columns (total 23 columns):
Series            6684 non-null int64
Court             6684 non-null int64
Round             6684 non-null int64
Best of           6684 non-null int64
1st Player        6684 non-null int32
2nd Player        6684 non-null int32
P1Rank            6684 non-null int32
P2Rank            6684 non-null int32
MaxP1             6684 non-null float64
MaxP2             6684 non-null float64
AvgP1             6684 non-null float64
AvgP2             6684 non-null float64
WElo              6684 non-null float64
WSurfElo          6684 non-null float64
WHand             6684 non-null int64
WBHand            6684 non-null float64
LElo              6684 non-null float64
LSurfElo          6684 non-null float64
LHand             6684 non-null int64
LBHand            6684 non-null float64
Surface__Clay     6684 non-null uint8
Surface__Grass    6684 non-null uint8
Surface__Hard     6684 non-null uint8

In [137]:
X.describe()

,Series,Court,Round,Best of,1st Player,2nd Player,P1Rank,P2Rank,MaxP1,MaxP2,...,WSurfElo,WHand,WBHand,LElo,LSurfElo,LHand,LBHand,Surface__Clay,Surface__Grass,Surface__Hard
count,6684.000000,6684.000000,6684.000000,6684.000000,6684.000000,6684.000000,6684.000000,6684.000000,6684.000000,6684.000000,...,6684.000000,6684.000000,6684.000000,6684.000000,6684.000000,6684.000000,6684.000000,6684.000000,6684.000000,6684.000000
mean,1.430580,0.168761,1.529623,3.397965,101.961251,101.961251,67.120287,67.120287,2.857102,2.857102,...,1737.341427,0.151706,1.813435,1789.398668,1737.341427,0.151706,1.813435,0.285757,0.132555,0.581688
std,1.500301,0.374569,2.082725,0.798530,63.159379,63.159379,83.225931,83.225931,3.142151,3.142151,...,153.533439,0.358762,0.389591,157.982929,153.533439,0.358762,0.389591,0.451808,0.339119,0.493319
min,0.000000,0.000000,0.000000,3.000000,0.000000,0.000000,1.000000,1.000000,1.010000,1.010000,...,1223.400000,0.000000,1.000000,1298.200000,1223.400000,0.000000,1.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,3.000000,43.750000,43.750000,26.000000,26.000000,1.530000,1.530000,...,1638.500000,0.000000,2.000000,1685.900000,1638.500000,0.000000,2.000000,0.000000,0.000000,0.000000
50%,1.000000,0.000000,1.000000,3.000000,101.000000,101.000000,51.000000,51.000000,2.010000,2.010000,...,1727.200000,0.000000,2.000000,1783.200000,1727.200000,0.000000,2.000000,0.000000,0.000000,1.000000
75%,2.000000,0.000000,2.000000,3.000000,157.000000,157.000000,88.000000,88.000000,2.950000,2.950000,...,1817.900000,0.000000,2.000000,1872.700000,1817.900000,0.000000,2.000000,1.000000,0.000000,1.000000
max,4.000000,1.000000,7.000000,5.000000,214.000000,214.000000,1921.000000,1921.000000,52.250000,52.250000,...,2163.500000,1.000000,2.000000,2203.400000,2163.500000,1.000000,2.000000,1.000000,1.000000,1.000000


Models construction

In [138]:
def baseline_model(X, Y):
    # This model will always predict the winner as the player with the highest rank.
    # It's the lower bound on accuracy that we wish to improve
    y_pred = (X['P1Rank'] > X['P2Rank']).astype(int)
    accuracy = round((y_pred == Y).sum()/len(Y), 2)
    return accuracy
    
print('Accuracy for the baseline model is:', baseline_model(X, Y))

Accuracy for the baseline model is: 0.37


In [139]:
def build_decision_tree(X_train, Y_train, X_valid, Y_valid):
    from sklearn.tree import DecisionTreeClassifier
    # Builds a decision tree and performs automatic hyper-parameters tuning
    scores = []
    for criterion in ('gini', 'entropy'):
        for depth in range(5, 50, 5):
            for leaves in range(10, 201, 30):
                dt = DecisionTreeClassifier(max_leaf_nodes=leaves,
                                            criterion=criterion,
                                            max_depth=depth)
                dt.fit(X_train, Y_train)
                valid_acc = round(accuracy_score(y_true=Y_valid, y_pred=dt.predict(X_valid)), 3)
                scores += [(valid_acc, criterion, depth, leaves)]
    best = max(scores)
    acc, criterion, depth, leaves = best
    print('Max accuracy on validation set:', acc)
    print('Criterion:', criterion)
    print('Max depth:', depth)
    print('Max leaves:', leaves)
    dt = DecisionTreeClassifier(max_leaf_nodes=leaves,
                                 criterion=criterion,
                                 max_depth=depth)
    dt.fit(pd.concat([X_train, X_valid]), np.concatenate([Y_train, Y_valid]))
    return dt, best

In [140]:
def build_bagging_classifier(X_train, Y_train, X_valid, Y_valid):
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.ensemble import BaggingClassifier
    scores = []
    for bootstrap in (True, False):
        for n_est in range(10, 201, 20):
            for max_samples in (0.25, 0.50, 0.75, 1.0):
                for criterion in ('gini', 'entropy'):
                    dt = DecisionTreeClassifier(criterion=criterion)
                    bagged_dt = BaggingClassifier(dt, bootstrap=bootstrap,
                                                  n_estimators=n_est,
                                                  max_samples=max_samples,
                                                  n_jobs=-1)
                    bagged_dt.fit(X_train, Y_train)
                    valid_acc = round(accuracy_score(y_true=Y_valid, y_pred=bagged_dt.predict(X_valid)), 3)
                    scores += [(valid_acc, bootstrap, n_est, max_samples, criterion)]
    best = max(scores)
    acc, bootsrap, n_est, max_samples, criterion = best
    print('Max accuracy on validation set:', acc)
    print('Boostrap:', bootsrap)
    print('N. estimators:', n_est)
    print('Max samples:', max_samples)
    print('Tree criterion:', criterion)
    bagged_dt = BaggingClassifier(dt, 
                                  bootstrap=bootstrap,
                                  n_estimators=n_est, 
                                  max_samples=max_samples)
    bagged_dt.fit(pd.concat([X_train, X_valid]), np.concatenate([Y_train, Y_valid]))
    return bagged_dt, best

In [141]:
def build_adaboost(X_train, Y_train, X_valid, Y_valid):
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.ensemble import AdaBoostClassifier
    scores = []
    for n_est in range(50, 301, 25):
        for learning_rate in (0.50, 0.75, 1.0, 1.5):
            for criterion in ('gini', 'entropy'):
                for depth in range(5, 26, 5):
                    for leaves in range(5, 100, 15):
                        dt = DecisionTreeClassifier(max_leaf_nodes=leaves,
                                                    criterion=criterion,
                                                    max_depth=depth)
                        boosted_dt = AdaBoostClassifier(dt,
                                                        n_estimators=n_est,
                                                        learning_rate=learning_rate)
                        boosted_dt.fit(X_train, Y_train)
                        valid_acc = round(accuracy_score(y_true=Y_valid, y_pred=bagged_dt.predict(X_valid)), 3)
                        scores += [(valid_acc, n_est, learning_rate, leaves, criterion, depth)]
    best = max(scores)
    acc, n_est, learning_rate, leaves, criterion, depth = best
    print('Max accuracy on validation set:', acc)
    print('N. estimators:', n_est)
    print('Learning rate:', learning_rate)
    print('Tree max leaves:', leaves)
    print('Tree max depth:', depth)
    print('Tree criterion:', criterion)
    dt = DecisionTreeClassifier(max_leaf_nodes=leaves,
                                criterion=criterion,
                                max_depth=depth)
    boosted_dt = AdaBoostClassifier(dt,
                                    n_estimators=n_est,
                                    learning_rate=learning_rate)
    boosted_dt.fit(pd.concat([X_train, X_valid]), np.concatenate([Y_train, Y_valid]))
    return boosted_dt, best

In [148]:
def build_random_forest(X_train, Y_train, X_valid, Y_valid):
    from sklearn.ensemble import RandomForestClassifier
    scores = []
    for n_est in range(50, 501, 50):
        for criterion in ('gini', 'entropy'):
            for bootstrap in (True, False):
                for n_features in (None, 'sqrt', 'log2'):
                    rf = RandomForestClassifier(n_estimators=n_est,
                                                bootstrap=bootstrap,
                                                criterion=criterion,
                                                max_features=n_features,
                                                n_jobs=-1)
                    rf.fit(X_train, Y_train)
                    valid_acc = round(accuracy_score(y_true=Y_valid, y_pred=rf.predict(X_valid)), 3)
                    scores += [(valid_acc, n_est, criterion, bootstrap, n_features)]
    best = max(scores)
    acc, n_est, criterion, bootstrap, features = best
    print('Max accuracy on validation set:', acc)
    print('N. estimators:', n_est)
    print('Criterion:', criterion)
    print('Bootstrap:', bootstrap)
    print('Features criterion (None means all features):', features)
    rf = RandomForestClassifier(n_estimators=n_est,
                               bootstrap=bootstrap,
                               criterion=criterion,
                               n_jobs=-1)
    rf.fit(pd.concat([X_train, X_valid]), np.concatenate([Y_train, Y_valid]))
    return rf, best

Let's split the data into train set, validation set and test set

In [149]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20)
X_train, X_valid, Y_train, Y_valid = train_test_split(X_train, Y_train, test_size=0.25)

In [144]:
dt, dt_params = build_decision_tree(X_train, Y_train, X_valid, Y_valid)

Max accuracy on validation set: 0.67
Criterion: gini
Max depth: 5
Max leaves: 160


In [145]:
bagged_dt, bagged_params = build_bagging_classifier(X_train, Y_train, X_valid, Y_valid)

Max accuracy on validation set: 0.675
Boostrap: False
N. estimators: 30
Max samples: 0.25
Tree criterion: entropy


In [146]:
boosted_dt, boosted_params = build_adaboost(X_train, Y_train, X_valid, Y_valid)

KeyboardInterrupt: 

In [150]:
rf, rf_params = build_random_forest(X_train, Y_train, X_valid, Y_valid)

Max accuracy on validation set: 0.683
N. estimators: 100
Criterion: entropy
Bootstrap: False
Features criterion (None means all features): sqrt


In [151]:
def report(X, Y, models):
    for model in models:
        print('Algorithm:', str(type(model)).split('.')[-1][:-2])
        rep = classification_report(y_true=Y, y_pred=model.predict(X))
        print(rep)
        print()

In [153]:
report(X_test, Y_test, [dt, bagged_dt, boosted_dt, rf])

Algorithm: DecisionTreeClassifier
              precision    recall  f1-score   support

           0       0.73      0.66      0.69       688
           1       0.67      0.74      0.70       649

    accuracy                           0.70      1337
   macro avg       0.70      0.70      0.70      1337
weighted avg       0.70      0.70      0.70      1337


Algorithm: BaggingClassifier
              precision    recall  f1-score   support

           0       0.79      0.84      0.81       688
           1       0.81      0.76      0.79       649

    accuracy                           0.80      1337
   macro avg       0.80      0.80      0.80      1337
weighted avg       0.80      0.80      0.80      1337


Algorithm: RandomForestClassifier
              precision    recall  f1-score   support

           0       0.67      0.66      0.66       688
           1       0.64      0.65      0.65       649

    accuracy                           0.65      1337
   macro avg       0.65      